## ewf-wfp-02-01-04 - Land Surface Temperature Anomalies Time Series

Land Surface Temperature Anomalies Time Series

---

### <a name="service">Service definition

In [1]:
service = dict([('title', 'Land Surface Temperature Anomalies Time Series'),
                ('abstract', 'Land Surface Temperature Anomalies Time Series'),
                ('id', 'ewf-wfp-02-01-04')])

### <a name="parameter">Parameter Definition 

In [2]:
N_1 = dict([('id', 'N_1'),
                          ('value', 'False'),
                          ('title', 'No Aggregation'),
                          ('abstract', 'No aggregation')])

In [3]:
N_3 = dict([('id', 'N_3'),
                          ('value', 'True'),
                          ('title', '30 Day Aggregation'),
                          ('abstract', 'Get a 30 day aggregation')])

In [4]:
N_6 = dict([('id', 'N_6'),
                          ('value', 'False'),
                          ('title', '60 Day Aggregation'),
                          ('abstract', 'Get a 30 day aggregation')])

In [5]:
 N_9 = dict([('id', 'N_9'),
                          ('value', 'False'),
                          ('title', '90 Day Aggregation'),
                          ('abstract', 'Get a 90 day aggregation')])

In [6]:
 N_12 = dict([('id', 'N_12'),
                          ('value', 'False'),
                          ('title', '120 Day Aggregation'),
                          ('abstract', 'Get a 120 day aggregation')])

In [7]:
 N_15 = dict([('id', 'N_15'),
                          ('value', 'False'),
                          ('title', '150 Day Aggregation'),
                          ('abstract', 'Get a 150 day aggregation')])

In [8]:
 N_18 = dict([('id', 'N_18'),
                          ('value', 'False'),
                          ('title', '180 Day Aggregation'),
                          ('abstract', 'Get a 180 day aggregation')])

In [9]:
 N_27 = dict([('id', 'N_27'),
                          ('value', 'False'),
                          ('title', '270 Day Aggregation'),
                          ('abstract', 'Get a 270 day aggregation')])

In [10]:
 N_36 = dict([('id', 'N_36'),
                          ('value', 'False'),
                          ('title', '360 Day Aggregation'),
                          ('abstract', 'Get a 360 day aggregation')])

In [11]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON((11.5030755518998 -11.1141633706909,41.0343255518998 -11.1141633706909,41.0343255518998 -34.9763656693858,11.5030755518998 -34.9763656693858,11.5030755518998 -11.1141633706909))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

In [12]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', 'SouthernAfrica'),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the MDOIS stack of products' identifiers

In [13]:
# 2015, 2016, 2017
input_identifiers = ('LST_SouthernAfrica_N3_averages_2015-01-05_2015-01-25.tif', 'LTA_LST_SouthernAfrica_N3_averages_1-5_1-25_2015_2017.tif')

#, 'LST_SouthernAfrica_N3_maxvalues_2015-01-01_2015-01-21.tif', 'LST_SouthernAfrica_N3_maxvalues_2016-01-01_2016-01-21.tif', 'LST_SouthernAfrica_N3_maxvalues_2017-01-01_2017-01-21.tif'



**Input references**

This is the MODIS stack catalogue references

In [14]:
input_references = ('https://catalog.terradue.com/modis/search?uid=MOD11C2.A2017001.006.2017013160908', 'https://catalog.terradue.com/modis/search?uid=MOD11C2.A2017009.006.2017020191355', 'https://catalog.terradue.com/modis/search?uid=MOD11C2.A2017017.006.2017025093208') 

**Data path**

This path defines where the data is staged-in. 

In [15]:
data_path = "/workspace/modis/outputs/output_02_03"

#### Aux folders

In [16]:
output_folder = ''

In [17]:
temp_folder = 'temp'

#### Import Modules

In [18]:
import os
import shutil

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads

import pdb

#### Auxiliary vars

In [19]:
check_results = False

#### Auxiliary methods

In [20]:
def rm_cfolder(folder):
    #folder = '/path/to/folder'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e) 
    
def get_metadata(filepath):
        
    #pdb.set_trace()
    ds = gdal.Open(filepath)
    projection = ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    no_data_value = ds.GetRasterBand(1).GetNoDataValue()
    data_type = ds.GetRasterBand(1).DataType
    return projection, geotransform, no_data_value, data_type


def get_matrix_list(image_list):
    mat_list = []
    for img in image_list:
        dataset = gdal.Open(img)
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
        
        print(type(product_array))
    return mat_list


def calc_anomaly(agg_file, LTA_file):
    
    
    #file_list = []
    #if not os.path.isdir('tmp_data'):
    #    os.mkdir('tmp_data')
    #for enclosure in dataframe['enclosure'].tolist():
    #    filepath = 'tmp_data/' + os.path.basename(enclosure)
    #    status = get_product(enclosure, filepath)
    #    if status == 200:
    #        file_list.append(filepath)
    #print(file_list)
    
    
    
    if agg_file and LTA_file:
        
        
        agg_and_LTA = get_matrix_list([agg_file, LTA_file])
        print('Aggregation and LTA converted to matrices')
        
        print(agg_and_LTA[0].dtype)
        print(agg_and_LTA[1].dtype)
        
        #anomaly_values = np.divide(agg_and_LTA[0] * 1.0, agg_and_LTA[1] * 1.0)
        
        anomaly_values = np.divide(agg_and_LTA[0] * 1.0, agg_and_LTA[1] * 1.0, out=np.zeros_like(agg_and_LTA[0] * 1.0), where=agg_and_LTA[1]!=0)
        
        #anomaly_values[(agg_and_LTA[0] == 0)] = 0
        #anomaly_values[(agg_and_LTA[1] == 0)] = 0
        
        print(anomaly_values.dtype)
        
        projection, geotransform, no_data_value, data_type = get_metadata(agg_file)
        
        
        
        #for file_ in file_list:
        #    os.remove(file_)
        
        return anomaly_values, projection, geotransform, no_data_value, data_type
    
    else:
        return None, None, None


def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    if mask is not None and mask is not 0:
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)

    

def write_anomaly_output(anomaly, output_folder, product_name, first_date, last_date, lta_start_year, lta_end_year, aggregation, mask_no_value, N_value, regionOfInterest, roi_name, projection, geo_transform, no_data_value):
    #image_number = (datetime.strptime(last_date, '%Y-%m-%d') - datetime.strptime(first_date, '%Y-%m-%d')).days
    
    
    #filename =  output_folder + '/' + product_name + '_Anomaly_' + roi_name + '_N' + str(N_value) + '_' + aggregation + '_' + first_date + '_' + last_date + '_LTA' + str(lta_start_year) + '_' + str(lta_end_year) + '.tif'
    
    filename = os.path.join(output_folder, product_name + '_Anomaly_' + roi_name + '_N' + str(N_value) + '_' + aggregation + '_' + first_date + '_' + last_date + '_LTA' + str(lta_start_year) + '_' + str(lta_end_year) + '.tif')
    
    write_output_image(filename, anomaly, 'GTiff', gdal.GDT_Float32, mask_no_value, projection, geo_transform, no_data_value)
    return filename



#### Auxiliary folders

In [21]:
#Create folders
#if not os.path.isdir(data_path):
#    os.mkdir(data_path)

if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

#### Workflow

In [22]:
#print('hello')
#pdb.set_trace()
if isinstance(input_identifiers, str):
    input_identifiers = [input_identifiers]

region_of_interest = regionOfInterest['value']
name_of_region = nameOfRegion['value']

filepath_agg = os.path.join(data_path, input_identifiers[0].split('/')[-1])
filepath_LTA = os.path.join(data_path, input_identifiers[1].split('/')[-1])

#filepath_agg = data_path + '/' + input_identifiers[0]
#filepath_LTA = data_path + '/' + input_identifiers[1]

# N time steps
nlist = [N_1['value'], N_3['value'], N_6['value']]
nlist = [n == 'True' for n in nlist]
nvalues = [1, 3, 6]

# list of files

print(filepath_agg)
print(filepath_LTA)

# get metadata from filenames

file_name_elements = os.path.basename(filepath_agg).split('.')[0].split('_')
#['LST', 'SouthernAfrica', 'N3', 'averages', '2015-01-01', '2015-01-21']
first_date = file_name_elements[-2]
last_date = file_name_elements[-1]
agg_type = file_name_elements[-3]
Nn = file_name_elements[-4]


file_name_elements = os.path.basename(filepath_LTA).split('.')[0].split('_')
#['LTA', 'LST', 'SouthernAfrica', 'N3', 'averages', '1-1', '1-21', '2015', '2017']

start_year = file_name_elements[-2]
end_year = file_name_elements[-1]

agg_type_LTA = file_name_elements[-5]
Nn_LTA = file_name_elements[-6]



# do some checks TODO
pass

for bl,nv in zip(nlist, nvalues):
    
    # only works for selected N time steps
    if bl:        

            #filename = os.path.splitext(os.path.basename(file_list[0]))[0].split('_')
            #agr = filename[3]
            #prod_type = filename[0]
            #N_value = filename[2]
            #region = filename[1]

        
            #agg_values = get_matrix_list(agg_file)[0]
        
            print(filepath_agg)
            print(filepath_LTA)
        
            anomaly_values, projection, geotransform, no_data_value, data_type = calc_anomaly(filepath_agg, filepath_LTA)
        
            filename = write_anomaly_output(anomaly_values, output_folder, 'LST', first_date, last_date, start_year, end_year, agg_type, None, nv, region_of_interest, name_of_region, projection, geotransform, no_data_value)
            
            
            
            

/workspace/modis/outputs/output_02_03/LST_SouthernAfrica_N3_averages_2015-01-05_2015-01-25.tif
/workspace/modis/outputs/output_02_03/LTA_LST_SouthernAfrica_N3_averages_1-5_1-25_2015_2017.tif
/workspace/modis/outputs/output_02_03/LST_SouthernAfrica_N3_averages_2015-01-05_2015-01-25.tif
/workspace/modis/outputs/output_02_03/LTA_LST_SouthernAfrica_N3_averages_1-5_1-25_2015_2017.tif
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Aggregation and LTA converted to matrices
uint16
uint16
float64


In [23]:
if check_results:

    import matplotlib
    import matplotlib.pyplot as plt

    fig = plt.figure()
    plt.imshow(anomaly_values)
    plt.show()

In [24]:
if check_results:

    import rasterio

    from rasterio.plot import show

    # Load data
    #raster = output_folder + '/' + 'LST_Anomaly_SouthernAfrica_N3_averages_2015-01-01_2015-01-21_LTA2015_2017.tif'
    raster = filename
    data = rasterio.open(raster)

    show(data)

    #data = georasters.from_file(raster)

    # Plot data
    #data.plot()

#### Remove temporay files and folders

In [25]:
rm_cfolder(temp_folder)

os.rmdir(temp_folder)